# Exercise 6

1. Using the EFIplus_medit.zip dataset, test if the frequency of sites with presence and absence of Salmo trutta fario (Brown Trout) are independent from the country. Please state which is/are the null hypothesis of your test(s). You may try to produce an alluvial plot.

In [8]:
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency, chi2
import plotly.graph_objects as go

In [3]:
file_path = 'EFIplus_medit.zip'  # Path to the zip file
df = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

df_trout = df[df['Salmo trutta fario'] == 1]
print(df_trout.head())

    Site_code   Latitude  Longitude Country Catchment_name  Galiza  Subsample  \
1  ES_02_0001  40.530188  -1.887796   Spain           Tejo       0          1   
2  ES_02_0002  40.595432  -1.928079   Spain           Tejo       0          1   
3  ES_02_0003  40.656184  -1.989831   Spain           Tejo       0          1   
4  ES_02_0004  40.676402  -2.036274   Spain           Tejo       0          1   
5  ES_02_0005  40.732830  -2.078003   Spain           Tejo       0          1   

   Calib_EFI_Medit  Calib_connect  Calib_hydrol  ...  Squalius malacitanus  \
1                1              1             1  ...                     0   
2                1              1             1  ...                     0   
3                1              1             1  ...                     0   
4                1              1             1  ...                     0   
5                1              1             1  ...                     0   

   Squalius pyrenaicus  Squalius torgalensis

Χ² test:

H0: A presença ou ausência de Salmo trutta fario é independente do país.

H1: A distribuição da truta castanha é igual entre os países considerados.

In [4]:
contingency_table = pd.crosstab(df['Country'], df['Salmo trutta fario'])
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Valor crítico para α = 0.05
alpha = 0.05
critical = chi2.ppf(q=1 - alpha, df=dof)

# Imprimir resultados
print(f'Estatística do Qui-quadrado: {chi2_stat:.4f}')
print(f'Graus de liberdade: {dof}')
print(f'Valor crítico (α = 0.05): {critical:.4f}')
print(f'p-valor: {p_val:.4f}')

# Interpretação manual
if chi2_stat > critical:
    print("Rejeita-se H0: a presença da truta depende do país.")
else:
    print("Não se rejeita H0: não há evidência de dependência entre presença da truta e o país.")


Estatística do Qui-quadrado: 496.3724
Graus de liberdade: 3
Valor crítico (α = 0.05): 7.8147
p-valor: 0.0000
Rejeita-se H0: a presença da truta depende do país.


In [5]:
# Mapear a coluna 'Salmo trutta fario' para 'Present' (1) e 'Absent' (0)
df['trout_presence'] = df['Salmo trutta fario'].map({1: 'Present', 0: 'Absent'})

# Contar combinações entre 'Country' e 'trout_presence'
grouped = df.groupby(['Country', 'trout_presence']).size().reset_index(name='count')

# Criar listas únicas para labels (nós)
countries = list(grouped['Country'].unique())
presence = list(grouped['trout_presence'].unique())
labels = countries + presence

# Mapear os índices dos países e das categorias de presença/ausência
country_index = {country: i for i, country in enumerate(countries)}
presence_index = {status: i + len(countries) for i, status in enumerate(presence)}

# Criar as ligações (source, target, value)
sources = [country_index[c] for c in grouped['Country']]
targets = [presence_index[p] for p in grouped['trout_presence']]
values = grouped['count'].tolist()

# Criar o gráfico alluvial (Sankey diagram)
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values
    ))])

# Adicionar título e exibir o gráfico
fig.update_layout(title_text="Distribuição de Presença/Ausência da Truta Castanha por País", font_size=12)
fig.show()

2. Run the non-parametric equivalent of the test you used in exercise 5.3 and compare with the ANOVA 
test (5.3: Test whether there are differences in the mean elevation in the upstream catchment 
(Elevation_mean_catch) among the eight most sampled catchments. For which pairs of catchments are 
these diferences significant? Please state which is/are the null hypothesis of your test(s))

In [6]:
catchment_counts = df['Catchment_name'].value_counts()
top_8_catchments = catchment_counts.head(8).index
filtered_df = df[df['Catchment_name'].isin(top_8_catchments)]
filtered_df = filtered_df[['Elevation_mean_catch', 'Catchment_name']]
elev = filtered_df.dropna()

print(elev)


      Elevation_mean_catch Catchment_name
1              1603.519424           Tejo
2              1578.678579           Tejo
3              1553.219128           Tejo
4              1539.684999           Tejo
5              1493.914010           Tejo
...                    ...            ...
4947            704.675069           Tejo
4948            702.673201           Tejo
4949            697.730192           Tejo
4950            679.784324           Tejo
4951            678.426582           Tejo

[3976 rows x 2 columns]


In [10]:
f_stat, anova_p_value = stats.f_oneway(
    *[elev[elev['Catchment_name'] == catchment]['Elevation_mean_catch'] for catchment in top_8_catchments]
)

kruskal_stat, kruskal_p_value = stats.kruskal(
    *[elev[elev['Catchment_name'] == catchment]['Elevation_mean_catch'] for catchment in top_8_catchments]
)

f_stat_rounded = round(f_stat, 2)
anova_p_value_rounded = round(anova_p_value, 2)

kruskal_stat_rounded = round(kruskal_stat, 2)
kruskal_p_value_rounded = round(kruskal_p_value, 2)

# Exibir os resultados arredondados
print(f"Resultado do teste ANOVA:")
print(f"F-statistic: {f_stat_rounded}, P-value: {anova_p_value_rounded}")
print("\n")

print(f"Resultado do teste Kruskal-Wallis:")
print(f"Estatística do teste: {kruskal_stat_rounded}, P-value: {kruskal_p_value_rounded}")

Resultado do teste ANOVA:
F-statistic: 227.95, P-value: 0.0


Resultado do teste Kruskal-Wallis:
Estatística do teste: 1335.37, P-value: 0.0


Ambos os testes (ANOVA e Kruskal-Wallis) indicam que há diferenças significativas nas distribuições de elevação entre os 8 catchments mais amostrados. O p-valor de 0.0 nos dois testes indica que é altamente improvável que as diferenças observadas tenham ocorrido por acaso. Em resumo, as elevações médias ou as distribuições de elevação variam de forma estatisticamente significativa entre os catchments.

3. Using the winequality_red.csv file in the examples folder of the github repository, test which wine 
parameters discriminate the best between wine quality scores categorized into two classes using value 
5 as the threshold value (quality>5=“good” and quality<5=“bad”).

In [12]:
df = pd.read_csv('winequality_red.csv')
df['quality_class'] = df['quality'].apply(lambda x: 'good' if x > 5 else 'bad')
print(df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality quality_class  
0      9.4        5           bad  
1      9.8    

In [13]:
parameters = df.drop(columns=['quality', 'quality_class'])

# Realizar o teste para cada parâmetro
for param in parameters.columns:
    good = df[df['quality_class'] == 'good'][param]
    bad = df[df['quality_class'] == 'bad'][param]
    
    # Testar a normalidade dos dados (Shapiro-Wilk)
    _, p_normal_good = stats.shapiro(good)
    _, p_normal_bad = stats.shapiro(bad)

    # Se ambos os grupos forem normais, usamos o teste t de Student
    if p_normal_good > 0.05 and p_normal_bad > 0.05:
        t_stat, p_value = stats.ttest_ind(good, bad)
        test_type = 't-test'
    else:
        # Caso contrário, usamos o teste Mann-Whitney U
        u_stat, p_value = stats.mannwhitneyu(good, bad)
        t_stat = u_stat
        test_type = 'Mann-Whitney U test'
    
    # Exibir os resultados para o parâmetro
    print(f"\nParâmetro: {param}")
    print(f"Teste: {test_type}")
    print(f"Estatística do teste: {t_stat:.2f}, P-value: {p_value:.4f}")

    # Determinar se a diferença é significativa
    if p_value < 0.05:
        print("Diferença significativa entre as classes.")
    else:
        print("Nenhuma diferença significativa entre as classes.")


Parâmetro: fixed acidity
Teste: Mann-Whitney U test
Estatística do teste: 347895.50, P-value: 0.0012
Diferença significativa entre as classes.

Parâmetro: volatile acidity
Teste: Mann-Whitney U test
Estatística do teste: 197208.00, P-value: 0.0000
Diferença significativa entre as classes.

Parâmetro: citric acid
Teste: Mann-Whitney U test
Estatística do teste: 376272.50, P-value: 0.0000
Diferença significativa entre as classes.

Parâmetro: residual sugar
Teste: Mann-Whitney U test
Estatística do teste: 323150.50, P-value: 0.5797
Nenhuma diferença significativa entre as classes.

Parâmetro: chlorides
Teste: Mann-Whitney U test
Estatística do teste: 254091.00, P-value: 0.0000
Diferença significativa entre as classes.

Parâmetro: free sulfur dioxide
Teste: Mann-Whitney U test
Estatística do teste: 298401.50, P-value: 0.0326
Diferença significativa entre as classes.

Parâmetro: total sulfur dioxide
Teste: Mann-Whitney U test
Estatística do teste: 245006.00, P-value: 0.0000
Diferença signi